In [1]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import nivapy

# OSPAR RID spatial data

There are some issues with the Norwegian spatial data supplied to OSPAR - see e-mail from Hans Fredrik received 04/05/2018 at 10.30 for details. 

Eleanor Dening from OSPAR has provided an Excel spreadsheet highlighting the missing information for Norway. This file lists 262 Norwegian catchment boundaries and 46 monitoring stations, most of which are missing information. Unfortuantely, I do not recognise either the catchment or station IDs provided in this file; they do not appear to relate to any of the IDs in the NIVA or NIBIO systems, so I'm not sure where they originally come from. My best guess is the following:

 * [NVE](https://www.nve.no/) define 262 main catchments (vassdragområder) on the Norwegian mainland. We estimate/model pollutant fluxes for each of these catchments, and then aggregate them to the OSPAR regions. I therefore suspect the 262 blank rows in Eleanor's spreadsheet refer to these catchments <br><br>
 
 * There are three groups of rivers in the Norwegian RID programme: the 11 main rivers ("RID_11"), the 36 tributary rivers ("RID_36"), and the 108 legacy rivers ("RID_108"). The *Monitoring-Stations* sheet of Eleanor's spreadsheet includes almost all of the RID_11 and RID_36 rivers; the only one that's missing is **Beiarelva** (but I assume this is a mistake/omission?).
 
There is not much I can do with the old codes in Eleanor's spreadsheet, but I can produce a new dataset, which can hopefully be used on the OSPAR ODIMS platform instead. This notebook performs basic spatial processing to tidy up the Norwegian data for OSPAR.

## 1. Station data for RID_11 and RID_36 rivers

In [2]:
# Read station data
in_xlsx = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\OSPAR_Spatial_Data\RID_Sites_List_OSPAR.xlsx')
df = pd.read_excel(in_xlsx, sheetname='RID_All')

# Get just data for RID_11 and RID_36
df = df.query('rid_group in ("rid_11", "rid_36")')

# Save
out_csv = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\OSPAR_Spatial_Data\RID_11_and_36_OSPAR.csv')
df.to_csv(out_csv, index=False, encoding='utf-8')
           
df.head()           

,station_id,station_code,station_name,rid_group,ospar_region,station_type,nve_vassdrag_nr,vassom,lat,lon,utm_north,utm_east,utm_zone
0,29615,VESENUM,Numedalslågen,rid_11,SKAGERAK,R,015.A1,15,59.086,10.070,6551828.42,561347.37,32
1,29821,HOREVOS,Vosso(Bolstadelvi),rid_11,NORTH SEA,R,062.C1,62,60.647,6.112,6726970.00,342124.00,32
2,29783,ROGEORR,Orreelva,rid_11,NORTH SEA,R,028.4A,28,58.731,5.529,6512861.00,646406.00,31
3,29613,TELESKI,Skienselva,rid_11,SKAGERAK,R,016.A221,16,59.199,9.611,6562920.00,534741.00,32
4,29614,VAGEOTR,Otra,rid_11,SKAGERAK,R,021.A11,21,58.187,7.954,6450006.00,438495.00,32


## 2. NVE catchment data

### 2.1. Read NVE shapefile

In [3]:
# Read NVE shapefile
shp_path = (r'C:\Data\James_Work\Useful_Stuff\GIS_Data\Norway'
            r'\NVE_Catchment_Data\Vassdragsomr.shp')
gdf = gpd.read_file(shp_path)

# Tidy
del gdf['objektType'], gdf['SHAPE_Leng'], gdf['SHAPE_Area']
gdf.columns = ['vassom', 'vass_name', 'a_land_km2', 'a_tot_km2', 'geometry']

gdf.head()

,vassom,vass_name,a_land_km2,a_tot_km2,geometry
0,001,Haldenvassdraget/Iddefjorden,2495,2507,"POLYGON ((11.53071753500007 60.06724793100005,..."
1,002,Glommavassdraget/Hvaler og Singlefjorden,42443,43114,"POLYGON ((11.79870108800003 62.89947015400003,..."
2,003,Mossevassdraget/kyst Onsøy-Son,854,1052,"POLYGON ((10.88390043200008 59.85863994500005,..."
3,004,Hølenelva/Drøbaksundet øst,204,227,"POLYGON ((10.65549781000004 59.67329887100004,..."
4,005,Nesodden og Bunnefjorden,279,368,"POLYGON ((10.74745449600005 59.83758076200007,..."


### 2.2. Assign catchments to OSPAR regions

In [4]:
# Convert text to ints
gdf['vassom'] = gdf['vassom'].astype(int)

# Assign catches to OSPAR regions
bins = [0, 23, 90, 170, 247, 315]
labels = ['SKAGERAK', 'NORTH SEA', 'NORWEGIAN SEA2', 'LOFOTEN-BARENTS SEA', 'N/A']
gdf['ospar_region'] = pd.cut(gdf['vassom'], bins=bins, labels=labels).astype(str)

# Save to new shp
out_shp = (r'C:\Data\James_Work\Staff\Oeyvind_K\Elveovervakingsprogrammet'
           r'\Data\gis\shapefiles\norway_main_catchments_ospar.shp')
gdf.to_file(out_shp)

gdf

,vassom,vass_name,a_land_km2,a_tot_km2,geometry,ospar_region
0,1,Haldenvassdraget/Iddefjorden,2495,2507,"POLYGON ((11.53071753500007 60.06724793100005,...",SKAGERAK
1,2,Glommavassdraget/Hvaler og Singlefjorden,42443,43114,"POLYGON ((11.79870108800003 62.89947015400003,...",SKAGERAK
2,3,Mossevassdraget/kyst Onsøy-Son,854,1052,"POLYGON ((10.88390043200008 59.85863994500005,...",SKAGERAK
3,4,Hølenelva/Drøbaksundet øst,204,227,"POLYGON ((10.65549781000004 59.67329887100004,...",SKAGERAK
4,5,Nesodden og Bunnefjorden,279,368,"POLYGON ((10.74745449600005 59.83758076200007,...",SKAGERAK
5,6,Nordmarkvassdraget/kyst Gjersjøelva-Bygdøy,391,414,"POLYGON ((10.51273370900003 60.22951381100006,...",SKAGERAK
6,7,Lysakerelva,203,212,"POLYGON ((10.45515715100004 60.15770805700004,...",SKAGERAK
7,8,Sandvikselva,279,311,"POLYGON ((10.40415494500007 60.11627493000003,...",SKAGERAK
8,9,Årosvassdraget/indre Oslofjorden vest,215,253,"POLYGON ((10.37868058100003 59.92009158200006,...",SKAGERAK
9,10,Hurumlandet øst og sør,114,191,"POLYGON ((10.46051805100007 59.68189574000007,...",SKAGERAK


## 3. Basic map

The map below shows the 262 Norwegian catchments (just coloured on a scale from 1 to 262 here), overlaid with the 47 (= 11 + 36) RID monitoring stations. Hopefully these layers can be used on ODIMS.

In [5]:
# Simplify polys for quick display
gdf['geometry'] = gdf.simplify(0.01)

# Plot catchments
map1 = nivapy.spatial.choropleth_from_gdf(gdf, 
                                          'vassom', 
                                          geom='geometry', 
                                          legend_name='Catchment ID')

# Overlay monitoring sites
map2 = nivapy.spatial.quickmap(df, 
                               lon_col='lon', 
                               lat_col='lat',
                               popup='station_name',
                               map_obj=map1)

map2